In [1]:
import pandas as pd

In [ ]:
names=[]
for i in range(0,3):
    names.append(f'B cell F')
    
for i in range(3,6):
    names.append(f'ittermate F')
    
for i in range(6,9):
    names.append(f'B cell M')
for i in range(9,12):
    names.append(f'ittermate M')
    
# for i in range(20,25):
#     names.append(f'BCH ma')
    
# for i in range(25,30):
#     names.append(f'delta ma')
print(names)
print(len(names))

In [ ]:
#data for 2 fold calculations
import numpy as np
from functools import reduce
df_x = pd.read_csv("Desktop/gish/scaled_data_genes - Sheet1.csv").drop(['name'],axis=1)
columns = df_x.columns
X = df_x.iloc[:,0:53]
X_np = X.to_numpy()
a_ref, b_ref ,ref , c_ref = X_np[0:3,:],X_np[3:6,:],X_np[6:9,:],X_np[9:12,:]
a_ref_avg,b_ref_avg ,ref_avg , c_ref_avg = np.average(a_ref,axis=0), np.average(b_ref,axis=0), np.average(ref,axis=0), np.average(c_ref,axis=0)
indices_a = np.where(ref_avg / a_ref_avg >= 1.25)
indices_b = np.where(ref_avg / a_ref_avg <= 0.75)
indices_c = np.where(ref_avg / b_ref_avg >= 1.25)
indices_d = np.where(ref_avg / b_ref_avg <= 0.75)
indices_e = np.where(ref_avg / c_ref_avg >= 1.25)
indices_f = np.where(ref_avg / c_ref_avg <= 0.75)
columns[indices_b],columns[indices_a]
final_indices = reduce(np.union1d, (indices_a,indices_b,indices_c,indices_d,indices_e,indices_f))
len(columns[final_indices]),columns[final_indices]
df_processed = df_x[columns[final_indices]]
df_processed

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns
import plotly.graph_objects as go
sns.set()
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
# import seaborn as sns
%matplotlib inline
plt.rcParams.update({'font.size': 14})

scaler = StandardScaler()


df = pd.read_csv("Desktop/gish/DEG log2 expressed genes for PCA - DEG log2 expressed genes for PCA.csv").drop(['name'],axis=1)
df.describe()
df.corr()
X = df_processed
ss = StandardScaler()
X = ss.fit_transform(X)
pca = PCA(3)
pca.fit(X)
scores = pca.transform(X)


# plot_variance= pca.explained_variance_ratio_
# print(plot_variance)
# plt.figure(figsize=(10,8))
# plt.plot(range(1,4),plot_variance.cumsum(), marker='o',linestyle='--')
# plt.xlabel("Number of clusters")
# plt.ylabel("wcss")
# plt.title("KMeans++ with PCA")
# plt.show()

# wcss=[]
# for i in range(1,8):
#     kmeans_pca = KMeans(n_clusters=i, init='k-means++',random_state=42)
#     kmeans_pca.fit(scores)
#     wcss.append(kmeans_pca.inertia_)


    
# print(scores.shape)   
# plt.figure(figsize=(10,8))
# plt.plot(range(1,8),wcss, marker='o',linestyle='--')
# plt.xlabel("Number of clusters")
# plt.ylabel("wcss")
# plt.title("KMeans++ with PCA")
# plt.show()

kmeans_pca = KMeans(algorithm='auto',copy_x=True,init='k-means++',max_iter=300,n_clusters = 2,n_init=10,random_state=42,tol=0.0001,verbose=0)#KMeans(n_clusters= 2 , init='k-means++',random_state=42)
kmeans_pca.fit(scores)
final_df = pd.concat([df_processed.reset_index(drop=True),pd.DataFrame(scores)], axis=1)
final_df.columns.values[-3:] = ['comp1','comp2','comp3']
final_df['kmeans_pca'] = kmeans_pca.labels_
final_df['segment'] = final_df['kmeans_pca'].map({0:'first',1:'second',2:'third'})
final_df['names'] = pd.DataFrame(names)
x_axis = final_df['comp2']
y_axis = final_df['comp1']
z_axis = final_df['comp3']
plt.figure(figsize=(10,8))
ax = plt.axes()
ax.set_facecolor("white")
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['top'].set_color('black')
ax.spines['right'].set_color('black')
sns.scatterplot(data = final_df, x = x_axis, y = y_axis, hue = final_df['segment'],palette=['g','r','c','m','b'])
for line in range(0,final_df.shape[0]):
     plt.text(final_df.comp2[line]+0.1, final_df.comp1[line]+0.1, final_df.names[line], horizontalalignment='left', size='small', color='black')
plt.title("Cluster by PCA Components", fontsize = 22)
plt.ylabel("Comp 1", fontweight='bold',fontsize = 18)
plt.xlabel("Comp 2", fontweight='bold',fontsize = 18)
plt.show()
print(final_df.head())

In [264]:
from sklearn.ensemble import ExtraTreesClassifier
targets = final_df["kmeans_pca"]
np.random.seed(42)
split_train = math.floor(0.75 * len(targets))
full_smaples = X
np.random.shuffle(full_smaples)  
train_x, test_x = full_smaples[0:split_train], full_smaples[split_train:]
targets = np.array(targets)
np.random.shuffle(targets) 
train_y, test_y = targets[0:split_train], targets[split_train:] 
model = ExtraTreesClassifier()
model.fit(train_x, train_y)
preds = model.predict(test_x)
gt = np.array(test_y)
gt = gt.flatten()
print((preds==gt).all())
count=0
for i,j in zip(preds,gt):
    print(i,j)
    if i==j:
        count=count + 1   

False
0 1
0 1
0 0


In [6]:
actual_df = final_df.iloc[:,0:24]
actual_df.head()
labels_df = final_df["kmeans_pca"]
actual_df['label'] = labels_df
actual_df.head()
labels_one = actual_df[(actual_df.label == 1)]
col = labels_one.pop("label")
labels_one.insert(0,col.name,col)
labels_one

,label,Bcl2-mRNA,Il4ra-mRNA,Ccl22-mRNA,Cxcr5-mRNA,Cd79a-mRNA,Il21r-mRNA,Cd19-mRNA,Ccr6-mRNA,Ifnz-mRNA,...,Cd22-mRNA,Il1rl1-mRNA,Cd28-mRNA,Cd79b-mRNA,Ntng2-mRNA,Lrg1-mRNA,Lax1-mRNA,Marcks-mRNA,Sell-mRNA,Gata3-mRNA
2,1,277.033053,1520.796030,457.874074,203.927108,1601.597337,530.018098,552.142265,475.188639,12.504964,...,442.483348,43.286415,197.193666,910.938546,220.279754,1176.428554,551.180345,1079.274602,1529.453313,134.668845
4,1,298.166576,1686.770914,536.699836,260.304153,1896.907358,693.828889,701.401373,552.791366,16.091529,...,449.616265,72.885163,232.853897,995.781707,207.296762,837.706094,644.607740,1062.040946,1501.245045,147.663447
5,1,288.713136,1510.045676,503.348559,189.942852,1322.951967,554.633129,478.655988,406.477704,21.843428,...,327.651420,63.630856,186.143995,772.117695,251.674279,1241.276540,464.410274,1009.546260,1341.946252,152.903996
9,1,321.575611,1865.845304,683.200932,184.935425,1624.369113,494.731709,534.781419,394.607435,29.448316,...,415.810222,57.718699,154.309176,643.151222,339.244601,1028.335196,415.810222,1993.062029,1671.486418,166.088502
10,1,326.040602,1821.854115,560.930068,295.656890,1869.766892,677.790498,735.052109,669.610268,31.552316,...,459.261493,70.116258,251.249926,1035.383416,257.092948,1280.790321,719.860253,1272.610091,1654.743699,162.435999
11,1,298.647654,1760.449327,598.277701,224.968134,2205.473627,545.228447,587.471371,528.527756,42.242925,...,422.429247,47.154893,183.707603,841.911313,283.911750,1127.787850,490.214405,1893.072463,1910.755548,134.587923


In [7]:
actual_df = final_df.iloc[:,0:24]
actual_df.head()
labels_df = final_df["kmeans_pca"]
actual_df['label'] = labels_df
actual_df.head()
labels_ZERO = actual_df[(actual_df.label == 0)]
col = labels_ZERO.pop("label")
labels_ZERO.insert(0,col.name,col)
labels_ZERO

,label,Bcl2-mRNA,Il4ra-mRNA,Ccl22-mRNA,Cxcr5-mRNA,Cd79a-mRNA,Il21r-mRNA,Cd19-mRNA,Ccr6-mRNA,Ifnz-mRNA,...,Cd22-mRNA,Il1rl1-mRNA,Cd28-mRNA,Cd79b-mRNA,Ntng2-mRNA,Lrg1-mRNA,Lax1-mRNA,Marcks-mRNA,Sell-mRNA,Gata3-mRNA
0,0,226.569105,1540.487929,542.309986,163.784895,1181.980992,440.399384,397.633328,358.506937,19.108238,...,280.254154,50.045385,149.226238,636.941258,241.127762,1227.476796,413.101902,1043.673747,1273.882516,160.145231
1,0,212.895425,1187.828551,487.009706,88.630284,743.763416,298.784566,387.414850,169.037140,21.015428,...,187.311425,63.046285,156.245140,499.801706,215.636568,1237.169121,303.353138,1110.162838,931.988555,131.574855
3,0,238.395238,1490.207683,508.133276,151.015310,1371.484955,379.912730,435.949858,388.460766,16.146291,...,308.679093,61.735819,152.914874,692.390950,236.495674,1191.026408,444.497894,1069.454335,1026.714153,150.065528
6,0,257.653546,1291.217960,483.837956,145.544751,1437.746121,384.513498,425.816738,336.326384,10.817515,...,271.421292,52.120755,130.793594,619.548602,196.682096,1324.653916,413.032401,1079.784707,1305.969117,103.258100
7,0,180.072544,1324.330987,397.755176,127.646360,1004.075387,406.872773,379.519982,242.756024,18.235194,...,235.917826,35.330689,127.646360,439.924063,202.866537,1542.013619,308.858604,1041.685475,1028.009080,101.433268
8,0,212.572904,1427.137178,373.935063,127.543742,1195.239465,398.091075,493.748881,334.319204,14.493607,...,311.129432,44.447062,136.239907,591.339169,244.458840,1338.243055,419.348365,989.430244,1135.332555,137.206147


In [8]:
def augment_return(data,aug_factor):    
    X = data.select_dtypes('float')
    label = data.label.unique().item()
    genes = X.columns.tolist()
    n, p = X.shape
    nnew = n * aug_factor
    mX = X.mean(axis=0)
    sdX = X.std(axis=0)
    idxnames = ['a{}_label{}'.format(i+1, label) for i in range(nnew)]
    colnames = X.columns.tolist()
    Z0 = pd.DataFrame(index=idxnames, columns=colnames)
    for i in range(nnew):
        w = np.concatenate((0, np.sort(np.random.uniform(size=n-1)), 1), axis=None)
        w = w[1:(n+1)] - w[0:n]
        Xt = X.sample(frac=1, replace=True).T
        Z0.iloc[i] = Xt.dot(w)
    mZ0 = Z0.mean(axis=0)
    sdZ0 = Z0.std(axis=0)
    for i in range(len(Z0)):
        Z0.iloc[i] = ((((Z0.iloc[i] - mZ0) / (sdZ0 + 1e-5)) * sdX) + mX)
    Z0.insert(0, 'label', [label]*nnew)
    frames = [data, Z0]
    final_label_frame = pd.concat(frames)
    return final_label_frame

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
import math

In [64]:
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
np.random.seed(42)
scaler = StandardScaler()
#ormalizer = Normalizer()
def feature_extraction(labels_one,labels_ZERO,aug_factor):    
    one = augment_return(labels_one,aug_factor)
    zero = augment_return(labels_ZERO,aug_factor)
    final_frames = [zero,one]
    final_frame_for_classifier = pd.concat(final_frames)
    X = final_frame_for_classifier.iloc[:,1:]
    X_new = np.array(X)
    np.random.shuffle(X_new)
    n, p = labels_one.shape
    total_samples = 2 *  n * (aug_factor + 1)
    split_train = math.floor(0.75 * total_samples)
    train_x, test_x = scaler.fit_transform(X_new[0:split_train]), scaler.transform(X_new[split_train:]) 
    Y = final_frame_for_classifier.iloc[:,0:1]
    Y = Y.reset_index(drop=True)
    Y = np.array(Y)
    Y = Y.flatten()
    np.random.shuffle(Y)
    train_y, test_y = Y[0:split_train] , Y[split_train:]
    model = ExtraTreesClassifier(n_estimators=100)
    model.fit(train_x, train_y)
    preds = model.predict(test_x)
    return preds,test_y
    #return X.columns, model.feature_importances_

In [ ]:
a = []
for i in range(20):
    print(i)
    preds, gt = feature_extraction(labels_one,labels_ZERO, 100)
    gt = np.array(gt)
    gt = gt.flatten()
    count=0
    for i,j in zip(preds,gt):
        if i==j:
            count=count + 1
    a.append(count/len(preds))
    print(count/len(preds))
a = np.array(a)
avr = np.average(a)
print(f"final-average-->{avr}")

In [ ]:
counter = {}
for i in range(20):
    col,values = feature_extraction(labels_one,labels_ZERO, 100)
    values = np.array(values)
    ind = np.argpartition(values, -10)[-10:]
    x = list(col[ind])
    for el in x:
        if counter.get(el) is None:
            counter[el] = 1
        else:
            counter[el] = counter[el] + 1
print(counter)